In [1]:
import pandas as pd
import pickle
import numpy as np
import tqdm

get_all_games = 1
get_features = 0
start_val = 500

In [4]:
if get_all_games:
    query = """
    SELECT *
    from `analytics-156605.barath.game_order_ludo`
    """

    all_games = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')
    all_games = all_games.to_dict('records')

    with open('../data/all_games_sep_1_to_sep_15.pkl', 'wb') as f:
        pickle.dump(all_games, f)

else:

    with open('../data/all_games_sep_1_to_sep_15.pkl', 'rb') as f:
        all_games = pickle.load(f)

Downloading:   0%|          | 0/9598569 [00:00<?, ?rows/s]

In [7]:
if get_features:
    query = """
    select *
    from `analytics-156605.barath.ludo_prediction_features`
    """

    other_features = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

    other_features = other_features.groupby('game_ref_id').apply(lambda g: g.to_dict('records')).to_dict()

    for name in tqdm.tqdm(other_features):
        other_features[name] = {item['user_id']:item for item in other_features[name]}

    with open('../data/other_features_sep_1_to_sep_22.pkl', 'wb') as f:
        pickle.dump(other_features, f)

else:

    with open('../data/other_features_sep_1_to_sep_15.pkl', 'rb') as f:
        other_features = pickle.load(f)

In [8]:
# query = """
#     select *
#     from `analytics-156605.barath.atleast_5_games`
#     """

# atleast_5_games = pd.read_gbq(query, use_bqstorage_api=True)

# atleast_5_games = atleast_5_games['game_ref_id'].to_list()

# atleast_5_games = {item:item for item in atleast_5_games}

In [9]:
with open('../data/q_val_data_sep_1_to_sep_5.pkl', 'rb') as f:
    results, results1 = pickle.load(f)

print(len(results) == len(results1))

for i in range(len(results)):
    results[i].update(results1[i])

print('1 done')

with open('../data/q_val_data_sep_6_to_sep_10.pkl', 'rb') as f:
    results1, results2 = pickle.load(f)

print(len(results1) == len(results2))

for i in range(len(results1)):
    results1[i].update(results2[i])

print('2 done')

with open('../data/q_val_data_sep_11_to_sep_15.pkl', 'rb') as f:
    results2, results3 = pickle.load(f)

print(len(results2) == len(results3))

for i in range(len(results2)):
    results2[i].update(results3[i])

print('3 done')

# with open('../data/q_val_data_sep_16_to_sep_22.pkl', 'rb') as f:
#     results3, results4 = pickle.load(f)

# print(len(results3) == len(results4))

# for i in range(len(results3)):
#     results3[i].update(results4[i])

# print('4 done')

results = results + results1
del results1
results = results + results2
del results2
# results = results + results3
del results3
# del results4

data = {item['division']:item for item in results if item is not None}
del results

True
1 done
True
2 done
True
3 done


In [10]:
for dat in tqdm.tqdm(data):
    if 'q_user_0' in data[dat]:
        temp1 = data[dat]['q_user_0'] #== data[dat]['q_max_user_0']
        temp2 = data[dat]['q_user_1'] #== data[dat]['q_max_user_1']
        # temp1 = (data[dat]['q_user_0'] - data[dat]['q_min_user_0']) / (data[dat]['q_max_user_0'] - data[dat]['q_min_user_0'])
        # temp2 = (data[dat]['q_user_1'] - data[dat]['q_min_user_1']) / (data[dat]['q_max_user_1'] - data[dat]['q_min_user_1'])
        # temp1 = temp1.astype('int32')
        # temp2 = temp2.astype('int32')
        if len(temp1) >= 25 and len(temp2) >= 25:
            if len(temp1) != len(temp2):
                length = np.min([len(temp1), len(temp2)])
                temp1 = temp1[:length]
                temp2 = temp2[:length]
            data[dat]['delta_q_0'] = np.mean(temp1 - temp2)
            data[dat]['delta_q_1'] = np.mean(temp2 - temp1)

100%|██████████| 6978165/6978165 [03:44<00:00, 31046.73it/s]


In [12]:
live_scores = {}
live_scores1 = {}

for d in tqdm.tqdm(all_games):
    if d['game_ref_id'] in data and d['game_ref_id'] in other_features and 'delta_q_1' in data[d['game_ref_id']]:
        dat = data[d['game_ref_id']]
        win = dat['score'][0] > dat['score'][1]
        pred = dat['delta_q_0'] > dat['delta_q_1']

        if dat['user_id_0'] not in live_scores:
            live_scores[dat['user_id_0']] = start_val
        if dat['user_id_1'] not in live_scores:
            live_scores[dat['user_id_1']] = start_val
        if dat['user_id_0'] not in live_scores1:
            live_scores1[dat['user_id_0']] = start_val
        if dat['user_id_1'] not in live_scores1:
            live_scores1[dat['user_id_1']] = start_val

        data[d['game_ref_id']]['q_score_0'] = live_scores[dat['user_id_0']]
        data[d['game_ref_id']]['q_score_1'] = live_scores[dat['user_id_1']]
        data[d['game_ref_id']]['score_score_0'] = live_scores1[dat['user_id_0']]
        data[d['game_ref_id']]['score_score_1'] = live_scores1[dat['user_id_1']]

        data[d['game_ref_id']]['dt'] = str(d['dt'])

        data[d['game_ref_id']]['hour'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_0']]['hour']

        data[d['game_ref_id']]['table_0'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_0']]['table']
        data[d['game_ref_id']]['table_1'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_1']]['table']

        data[d['game_ref_id']]['wallet_0'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_0']]['wallet']
        data[d['game_ref_id']]['wallet_1'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_1']]['wallet']

        data[d['game_ref_id']]['deposit_0'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_0']]['deposit']
        data[d['game_ref_id']]['deposit_1'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_1']]['deposit']

        data[d['game_ref_id']]['withdrawal_0'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_0']]['withdrawal']
        data[d['game_ref_id']]['withdrawal_1'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_1']]['withdrawal']

        data[d['game_ref_id']]['stw_0'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_0']]['stw']
        data[d['game_ref_id']]['stw_1'] = other_features[d['game_ref_id']][data[d['game_ref_id']]['user_id_1']]['stw']

        if win == pred:
            live_scores[dat['user_id_0']] += dat['delta_q_0']
            live_scores[dat['user_id_1']] += dat['delta_q_1']

        live_scores1[dat['user_id_0']] += (dat['score'][0] - dat['score'][1])/np.min([len(dat['q_user_0']), len(dat['q_user_1'])])
        live_scores1[dat['user_id_1']] += (dat['score'][1] - dat['score'][0])/np.min([len(dat['q_user_0']), len(dat['q_user_1'])])

100%|██████████| 9598569/9598569 [04:56<00:00, 32333.12it/s]


In [13]:
win = []
q_pred = []
skill_pred = []
q_score_0 = []
q_score_1 = []

feats = []
feats_test = []
y = []
y_test = []

which_feats = 0 # 0 - skill, 1 - game + skill, 2 - q + skill, 3 - q + game + skill, 4 - 3 + other_feats, 5 - 4 + more_features
five_game_filter = 0
under_sample = 0
over_sample = 0

for dat in tqdm.tqdm(data):
    if 'wallet_1' in data[dat] and 'dt' in data[dat]:
        # if not (five_game_filter and dat not in atleast_5_games):

        if data[dat]['dt'] in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15']:
            win.append(data[dat]['score'][0] > data[dat]['score'][1])
            q_pred.append(data[dat]['q_score_0'] > data[dat]['q_score_1'])
            skill_pred.append((data[dat]['mu_0'] - 3*data[dat]['sigma_0']) > (data[dat]['mu_1'] - 3*data[dat]['sigma_1']))

            q_score_0.append(data[dat]['score_score_0'])
            q_score_1.append(data[dat]['score_score_1'])

        if data[dat]['dt'] in ['2022-09-11', '2022-09-12', '2022-09-13']:

            if which_feats == 0:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1']])
            elif which_feats == 1:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['score_score_1']])
            elif which_feats == 2:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1']])
            elif which_feats == 3:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1']])
            elif which_feats == 4:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1'],
                            data[dat]['table_0'], data[dat]['table_1'], data[dat]['wallet_0'], data[dat]['wallet_1'], data[dat]['hour']])
            elif which_feats == 5:
                feats.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1'],
                            data[dat]['table_0'], data[dat]['table_1'], data[dat]['wallet_0'], data[dat]['wallet_1'], 
                            data[dat]['deposit_0'], data[dat]['deposit_1'], data[dat]['withdrawal_0'], data[dat]['withdrawal_1'],
                            data[dat]['stw_0'], data[dat]['stw_1'], data[dat]['hour']])

            y.append(data[dat]['score'][0] > data[dat]['score'][1])

        elif data[dat]['dt'] in ['2022-09-14', '2022-09-15']:

            if which_feats == 0:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1']])
            elif which_feats == 1:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['score_score_1']])
            elif which_feats == 2:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1']])
            elif which_feats == 3:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1']])
            elif which_feats == 4:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1'],
                            data[dat]['table_0'], data[dat]['table_1'], data[dat]['wallet_0'], data[dat]['wallet_1'], data[dat]['hour']])
            elif which_feats == 5:
                feats_test.append([data[dat]['mu_0'], data[dat]['sigma_0'], data[dat]['q_score_0'], data[dat]['score_score_0'], 
                            data[dat]['mu_1'], data[dat]['sigma_1'], data[dat]['q_score_1'], data[dat]['score_score_1'],
                            data[dat]['table_0'], data[dat]['table_1'], data[dat]['wallet_0'], data[dat]['wallet_1'], 
                            data[dat]['deposit_0'], data[dat]['deposit_1'], data[dat]['withdrawal_0'], data[dat]['withdrawal_1'],
                            data[dat]['stw_0'], data[dat]['stw_1'], data[dat]['hour']])

            y_test.append(data[dat]['score'][0] > data[dat]['score'][1])

100%|██████████| 6978165/6978165 [00:25<00:00, 274333.82it/s]


In [14]:
win = np.array(win).astype('int32')
q_pred = np.array(q_pred).astype('int32')
skill_pred = np.array(skill_pred).astype('int32')

feats = np.array(feats)
feats_test = np.array(feats_test)
y = np.array(y)
y_test = np.array(y_test)

In [36]:
feats_test.shape

(962151, 4)

In [15]:
skill_1 = feats_test[:,0] - 3*feats_test[:,1]
skill_2 = feats_test[:,2] - 3*feats_test[:,3]
bins = np.abs(skill_1 - skill_2)

In [16]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i, bins<(i+1))
    temp = y_test[idx]
    u1.append(np.sum(temp[skill_1[idx]>skill_2[idx]]) + np.sum(temp[skill_1[idx]<skill_2[idx]]==0))
    tot.append(len(skill_1[idx]))

In [17]:
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,232584,468258
1,60178,123976
2,22204,46789
3,15148,33151
4,17504,38356
5,37444,82140
6,10785,23865
7,4416,10092
8,2540,5756
9,2007,4633


In [18]:
if under_sample:

    if which_feats == 5:

        from imblearn.under_sampling import RandomUnderSampler
    
        def undersample(feats, y, limit):
            skill_1 = feats[:,0] - 3*feats[:,1]
            skill_2 = feats[:,4] - 3*feats[:,5]
            bins = skill_1 - skill_2

            a_bins = np.ceil(bins)

            feats = feats[np.logical_and(a_bins >= -1*limit, a_bins <= limit)]
            y = y[np.logical_and(a_bins >= -1*limit, a_bins <= limit)]
            a_bins = a_bins[np.logical_and(a_bins >= -1*limit, a_bins <= limit)]

            a_bins = np.expand_dims(a_bins, -1)

            under_sampler = RandomUnderSampler(random_state=42)
            _, a_bins = under_sampler.fit_resample(a_bins, a_bins)

            indices = under_sampler.sample_indices_
            feats = feats[indices]
            y = y[indices]

            return feats, y

        feats, y = undersample(feats, y, 20)
        # feats_test, y_test = undersample(feats, y)

In [19]:
if over_sample:

    if which_feats == 5:

        from imblearn.combine import SMOTETomek
    
        def oversample(feats, y, limit):
            skill_1 = feats[:,0] - 3*feats[:,1]
            skill_2 = feats[:,4] - 3*feats[:,5]
            bins = skill_1 - skill_2

            a_bins = np.ceil(bins)

            feats = feats[np.logical_and(a_bins >= -1*limit, a_bins <= limit)]
            y = y[np.logical_and(a_bins >= -1*limit, a_bins <= limit)]
            a_bins = a_bins[np.logical_and(a_bins >= -1*limit, a_bins <= limit)]
            
            a_bins = np.core.defchararray.add(y.astype('int32').astype('str'), a_bins.astype('int32').astype('str'))

            under_sampler = SMOTETomek(random_state=42, n_jobs=8)
            feats, a_bins = under_sampler.fit_resample(feats, a_bins)

            y = np.array([i[0] for i in a_bins]).astype('float32')

            return feats, y, 

        feats, y = oversample(feats, y, 20)
        # feats_test, y_test = undersample(feats, y)

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

In [21]:
test_binned = 0
train_binned = 0

limit = 25

In [22]:
def bin_set(feats, y, limit):
    skill_1 = feats[:,0] - 3*feats[:,1]
    skill_2 = feats[:,4] - 3*feats[:,5]
    bins = skill_1 - skill_2

    idx = np.logical_or(np.logical_and(bins<=limit, bins>=(limit-5)), np.logical_and(bins>=(-1*limit), bins<=(-1*limit+5)))

    return feats[idx], y[idx]

if train_binned:
    feats, y = bin_set(feats, y, limit)
if test_binned:
    feats_test, y_test = bin_set(feats_test, y_test, limit)

In [23]:
clf = LGBMClassifier(random_state=42, verbose=1)
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', GradientBoostingClassifier(random_state=42, verbose=1))])
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', LogisticRegression(random_state=42, verbose=1))])
clf = clf.fit(feats, y)

[LightGBM] [Info] Number of positive: 790048, number of negative: 658346
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1448394, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.545465 -> initscore=0.182363
[LightGBM] [Info] Start training from score 0.182363


In [24]:
y_pred = clf.predict(feats_test)
y_probs = clf.predict_proba(feats_test)

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

def perf(ytrue, ypred):
    accu = accuracy_score(ytrue, ypred)
    prec = precision_score(ytrue, ypred)
    rec = recall_score(ytrue, ypred)
    return accu*100, prec*100, rec*100

In [26]:
acc, precision, recall = perf(y_test, y_pred)
print(acc, precision, recall)

55.26970298840826 55.44805231172614 92.28006634912423


In [27]:
bins = np.abs(y_probs[:,1]-0.5)

In [28]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*0.05, bins<(i+1)*0.05)
    temp = y_test[idx]
    u1.append(np.sum(temp[y_probs[:,1][idx]>=0.5]==1) + np.sum(temp[y_probs[:,1][idx]<0.5]==0))
    tot.append(len(y_test[idx]))

In [29]:
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,287634,539920
1,197935,349190
2,33411,53884
3,10999,16543
4,1758,2554
5,40,59
6,1,1
7,0,0
8,0,0
9,0,0


In [30]:
with open('../models/ludo_skill_game_score_model.pkl', 'wb') as f:
    pickle.dump(clf, f)